## do

In [1]:
MODEL_NAME = 'blending6'
OOF_FILE = ['keras85','keras101','keras93','keras107']#pri-85,86,93,94,pub-85,93,94,101

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import scipy
import sklearn
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt
from matplotlib import rcParams
from tqdm import tqdm
import os

In [4]:
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)
SUBMISSION_NAME = f"/content/drive/MyDrive/Colab Notebooks/kaggle/MSCI/cite_sub/{MODEL_NAME}.csv"

In [5]:
df_list = ['']*len(OOF_FILE)
def std(x):
    return (x - np.mean(x)) / np.std(x)
for i,name in enumerate(OOF_FILE):
    df_list[i] = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/kaggle/MSCI/cite_oof/{name}.csv',index_col=0).apply(std,axis=1)
oof_df = pd.concat(df_list,axis=1)
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/kaggle/MSCI/dataset"
FP_CITE_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_cite_targets.h5")
test_list = ['']*len(OOF_FILE)
for i,name in enumerate(OOF_FILE):
    tn = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/kaggle/MSCI/cite_sub/{name}.csv',index_col=0).drop(columns='row_id').to_numpy().reshape(-1,140)
    test_list[i] = pd.DataFrame(data=tn,columns=oof_df.columns.unique()).apply(std,axis=1)
test_df = pd.concat(test_list,axis=1)
Y = pd.read_hdf(FP_CITE_TRAIN_TARGETS)
oof_df.shape,test_df.shape

((70988, 560), (48663, 560))

In [6]:
for i in range(len(OOF_FILE)):
    print(OOF_FILE[i],correlation_score(Y,df_list[i]))

keras85 0.8946317531926749
keras101 0.8946940477778914
keras93 0.8945484302429195
keras107 0.8947868817349912


In [8]:
weights = [0.5,0.5,0.5,1]
train_pred = 0
for i,w in enumerate(weights):
    train_pred += df_list[i] * w
train_pred = train_pred.fillna(0)
correlation_score(Y,train_pred)

0.8951184015217777

In [9]:
test_pred = 0
for i,w in enumerate(weights):
    test_pred += test_list[i] * w
test_pred = test_pred.fillna(0)
test_pred = test_pred.values
sub = pd.DataFrame(test_pred.ravel())
sub['row_id'] = sub.index
sub = sub.rename(columns={0:'target'})
sub.to_csv(SUBMISSION_NAME)